In [1]:
import os
import json
from glob import glob
import numpy as np
import paprika.io as io
import matplotlib.pyplot as plt

import simtk.unit as unit
from simtk.openmm import XmlSerializer, Platform
from simtk.openmm.app import PDBFile, Simulation
from simtk.openmm.openmm import CustomNonbondedForce, LangevinIntegrator, MonteCarloBarostat
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion, AlchemicalState

from MDAnalysis.coordinates.DCD import DCDFile

In [2]:
def get_simulation_object(sel1, sel2, system_xml, coordinates, pme_treatment="direct-space"):
    # Setup Alchemical system
    factory = AbsoluteAlchemicalFactory(
        consistent_exceptions=True,
        disable_alchemical_dispersion_correction=False,
        alchemical_pme_treatment=pme_treatment,
        split_alchemical_forces=True,
    )
    alchemical_region = AlchemicalRegion(
        alchemical_atoms=sel1,
        annihilate_electrostatics=True,
        annihilate_sterics=True,
    )
    system_alch = factory.create_alchemical_system(
        system_xml,
        alchemical_region,
    )
    
    # Set Interaction groups
    alchemical_state = AlchemicalState.from_system(system_alch)
    for force in system_alch.getForces():
        if isinstance(force, CustomNonbondedForce):
            force.setInteractionGroupParameters(0, sel1, sel2)
            
    # Create Simulation object
    simulation = Simulation(
        coordinates.topology, 
        system_alch, 
        LangevinIntegrator(298.15, 1.0/unit.picoseconds, 4.0*unit.femtosecond),
        Platform.getPlatformByName('CUDA'),
        {'CudaPrecision': 'mixed'},
    )
    alchemical_state.lambda_sterics = 1.0
    alchemical_state.lambda_electrostatics = 1.0
    alchemical_state.apply_to_context(simulation.context)
    
    return simulation

In [3]:
def find_nearest(array, value):
    return np.abs(array - value).argmin()

## Get indices for host and guest molecules

In [4]:
coords = PDBFile("windows/p000/restrained.pdb")

In [5]:
host_resname = "CB7"
guest_resname = "AMT"
solvent_resname = ["HOH", "Na+", "Cl-"]
host_indices = []
guest_indices = []
solvent_indices = []
for atom in coords.getTopology().atoms():
    if atom.residue.name == host_resname:
        host_indices.append(atom.index)
    if atom.residue.name == guest_resname:
        guest_indices.append(atom.index)
    if atom.residue.name in solvent_resname:
        solvent_indices.append(atom.index)

Get `G1` and `G2` indices

In [6]:
atom1 = None
atom2 = None
for atom in coords.topology.atoms():
    if atom.residue.name == "DM1":
        atom1 = atom.index
    if atom.residue.name == "AMT" and atom.name == "C5":
        atom2 = atom.index

## Generate simulation objects for each pair

Load topology from p000

In [7]:
with open("windows/p000/restrained.xml", "r") as f:
    system = XmlSerializer.deserialize(f.read())

In [8]:
pme_treatment = "direct-space"

Four pair-interactions (Note: complex = host + guests, solvent = water + ions):

1. host-guest
2. host-solvent
3. guest-solvent
4. complex-solvent

In [9]:
simulation_hg = get_simulation_object(host_indices, guest_indices, system, coords, pme_treatment)
simulation_hw = get_simulation_object(host_indices, solvent_indices, system, coords, pme_treatment)
simulation_gw = get_simulation_object(guest_indices, solvent_indices, system, coords, pme_treatment)
simulation_hgw = get_simulation_object((host_indices+guest_indices), solvent_indices, system, coords, pme_treatment)

## Get nonbonded pair-potential energy 

In [10]:
with open('windows.json', 'r') as f:
    json_data = f.read()
windows = json.loads(json_data, object_hook=io.json_numpy_obj_hook)
windows_list = windows['pull']
n_windows = len(windows_list)

In [11]:
dr = 0.1
r_distance = np.arange(5.5, 24.5+dr, dr)
n_counts  = np.zeros(len(r_distance))
skip_frames = 10

In [12]:
pair_interaction = {
    "host_guest": {
        "elec": np.zeros(len(r_distance)), 
        "vdw": np.zeros(len(r_distance)), 
        "total": np.zeros(len(r_distance)),
    },
    "host_solvent": {
        "elec": np.zeros(len(r_distance)), 
        "vdw": np.zeros(len(r_distance)), 
        "total": np.zeros(len(r_distance)),
    },
    "guest_solvent": {
        "elec": np.zeros(len(r_distance)), 
        "vdw": np.zeros(len(r_distance)), 
        "total": np.zeros(len(r_distance)),
    },
    "host_guest_solvent": {
        "elec": np.zeros(len(r_distance)),
        "vdw": np.zeros(len(r_distance)),
        "total": np.zeros(len(r_distance)),
    },
}

In [ ]:
for i in range(n_windows):
    window = windows_list[i]
    print(f"Analyzing window {window}")
    
    for trajectory in glob(f"windows/{window}/production-v*.dcd"):
        with DCDFile(trajectory) as dcd:
            header = dcd.header

            for frame in dcd.readframes()[0]:#[::skip_frames]:
                r_instance = np.linalg.norm(frame[atom1] - frame[atom2])
                idx = find_nearest(r_distance, r_instance)
                n_counts[idx] += 1
                
                simulation_hg.context.setPositions(frame * unit.angstrom)
                simulation_hw.context.setPositions(frame * unit.angstrom)
                simulation_gw.context.setPositions(frame * unit.angstrom)
                simulation_hgw.context.setPositions(frame * unit.angstrom)

                pair_interaction["host_guest"]["elec"][idx]         += simulation_hg.context.getState(getEnergy=True, groups={1}).getPotentialEnergy() / unit.kilocalorie_per_mole
                pair_interaction["host_guest"]["vdw"][idx]          += simulation_hg.context.getState(getEnergy=True, groups={2}).getPotentialEnergy() / unit.kilocalorie_per_mole
                pair_interaction["host_solvent"]["elec"][idx]       += simulation_hw.context.getState(getEnergy=True, groups={1}).getPotentialEnergy() / unit.kilocalorie_per_mole
                pair_interaction["host_solvent"]["vdw"][idx]        += simulation_hw.context.getState(getEnergy=True, groups={2}).getPotentialEnergy() / unit.kilocalorie_per_mole
                pair_interaction["guest_solvent"]["elec"][idx]      += simulation_gw.context.getState(getEnergy=True, groups={1}).getPotentialEnergy() / unit.kilocalorie_per_mole
                pair_interaction["guest_solvent"]["vdw"][idx]       += simulation_gw.context.getState(getEnergy=True, groups={2}).getPotentialEnergy() / unit.kilocalorie_per_mole
                pair_interaction["host_guest_solvent"]["elec"][idx] += simulation_hgw.context.getState(getEnergy=True, groups={1}).getPotentialEnergy() / unit.kilocalorie_per_mole
                pair_interaction["host_guest_solvent"]["vdw"][idx]  += simulation_hgw.context.getState(getEnergy=True, groups={2}).getPotentialEnergy() / unit.kilocalorie_per_mole

# Average potential energies
n_counts[n_counts == 0] = 1 
pair_interaction["host_guest"]["elec"] /= n_counts
pair_interaction["host_guest"]["vdw"] /= n_counts

pair_interaction["host_solvent"]["elec"] /= n_counts
pair_interaction["host_solvent"]["vdw"] /= n_counts

pair_interaction["guest_solvent"]["elec"] /= n_counts
pair_interaction["guest_solvent"]["vdw"] /= n_counts

pair_interaction["host_guest_solvent"]["elec"] /= n_counts
pair_interaction["host_guest_solvent"]["vdw"] /= n_counts

# Shift energy to zero
for pair in pair_interaction.keys():
    pair_interaction[pair]["elec"] -= pair_interaction[pair]["elec"][-1]
    pair_interaction[pair]["vdw"] -= pair_interaction[pair]["vdw"][-1]
    pair_interaction[pair]["total"] = pair_interaction[pair]["elec"] + pair_interaction[pair]["vdw"]

# Save results to JSON
with open("pair_interaction.json", "w") as f:
    dumped = json.dumps(pair_interaction, cls=io.NumpyEncoder)
    f.write(dumped)

Analyzing window p000
Analyzing window p001


## Plot pair-potential profile

In [ ]:
plt.figure(figsize=(12,10))
plt.subplot(2,2,1)
plt.plot(r_distance, pair_interaction["host_guest"]["elec"], 'o-', label="elec")
plt.plot(r_distance, pair_interaction["host_guest"]["vdw"], 'o-', label="vdw")
plt.ylabel(r"$\Delta$G (kcal/mol)", fontsize=14)
plt.title("Host-Guest", fontsize=14)

plt.subplot(2,2,3)
plt.plot(r_distance, pair_interaction["host_guest_solvent"]["elec"], 'o-', label="elec")
plt.plot(r_distance, pair_interaction["host_guest_solvent"]["vdw"], 'o-', label="vdw")
plt.xlabel(r"r ($\AA$)", fontsize=14)
plt.ylabel(r"$\Delta$G (kcal/mol)", fontsize=14)
plt.title("Host-Guest complex with solvent", fontsize=14)

plt.subplot(2,2,2)
plt.plot(r_distance, pair_interaction["guest_solvent"]["elec"], 'o-', label="elec")
plt.plot(r_distance, pair_interaction["guest_solvent"]["vdw"], 'o-', label="vdw")
plt.title("Guest-solvent", fontsize=14)

plt.subplot(2,2,4)
plt.plot(r_distance, pair_interaction["host_solvent"]["elec"], 'o-', label="elec")
plt.plot(r_distance, pair_interaction["host_solvent"]["vdw"], 'o-', label="vdw")
plt.xlabel(r"r ($\AA$)", fontsize=14)
plt.title("Host-solvent", fontsize=14)

plt.legend(fontsize=12)